# DATA PREPOCESSING

1. data understanding tambahkan identifikasi outlier
2. tambahkan menu preposesing isinya penghapusan outlier yang mengunakan 3 motode

# Data Understanding dan Preprocessing dengan Deteksi Outlier
import pandas as pd

- Baca dataset
df = pd.read_excel("data_iris.xlsx")

print("Shape:", df.shape)
display(df.head())
df.info()
display(df.describe())

- Encode kolom kategori jika ada
from sklearn.preprocessing import LabelEncoder
if 'Class' in df.columns:
    le = LabelEncoder()
    df['Class'] = le.fit_transform(df['Class'])

- Identifikasi outlier per fitur numerik pakai IQR
- Exclude 'Class' from numerical features for IQR outlier detection
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
numerical_features_iqr = [col for col in numerical_features if col != 'Class']


def detect_outlier_iqr(df, feature):
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[feature + '_outlier'] = ((df[feature] < lower) | (df[feature] > upper)).astype(int)

for col in numerical_features_iqr:
    detect_outlier_iqr(df, col)

display(df.head())

Penjelasan:
Dataset yang digunakan adalah data_iris.xlsx dengan bentuk (shape) sesuai jumlah baris dan kolom yang ditampilkan. Pada tahap awal, data divisualisasikan menggunakan head() sehingga terlihat sampel 5 baris pertama, kemudian ditampilkan informasi struktur dataset melalui info() yang menunjukkan tipe data setiap kolom, jumlah non-null, serta ada tidaknya nilai yang hilang. Setelah itu, ditampilkan juga statistik deskriptif menggunakan describe() yang mencakup nilai minimum, maksimum, mean, standar deviasi, dan kuartil dari setiap kolom numerik. Untuk memastikan konsistensi, jika terdapat kolom kategori seperti Class, kolom tersebut di-encode ke bentuk numerik menggunakan LabelEncoder. Selanjutnya, dilakukan identifikasi outlier pada setiap fitur numerik (selain kolom target/kelas) dengan metode IQR (Interquartile Range). Proses ini menghitung kuartil bawah (Q1), kuartil atas (Q3), dan jarak antarkuartil (IQR), lalu menentukan batas bawah (Q1 – 1.5×IQR) dan batas atas (Q3 + 1.5×IQR). Setiap nilai yang berada di luar rentang tersebut ditandai sebagai outlier dengan nilai 1, sedangkan nilai yang masih berada dalam batas dianggap normal dengan nilai 0. Hasil akhir menambahkan kolom baru untuk setiap fitur numerik, misalnya sepal_length_outlier atau petal_width_outlier, sehingga dapat langsung dilihat fitur mana yang mengandung anomali. Dengan cara ini, data understanding menghasilkan gambaran menyeluruh mengenai struktur dataset, ringkasan statistik, serta distribusi nilai ekstrem (outlier) pada setiap fitur numerik, yang nantinya akan sangat penting dalam tahap preprocessing maupun pemodelan.

- Install Python 3.11
!sudo apt-get update
!sudo apt-get install python3.11 python3.11-distutils -y

- Set python3 default ke versi 3.11
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1

- Upgrade pip
!python3 -m ensurepip --upgrade
!python3 -m pip install --upgrade pip

!pip install pycaret

import pandas as pd
from pyod.models.abod import ABOD
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from sklearn.preprocessing import LabelEncoder

- Data Preparation

- Baca dataset
df = pd.read_excel("data_iris.xlsx")

- Encode kolom kategori jika ada
if 'Class' in df.columns:
    le = LabelEncoder()
    df['Class'] = le.fit_transform(df['Class'])

- Ambil hanya fitur numerik
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
X = df[numerical_features]


- Fungsi deteksi outlier PyOD
def detect_outlier_pyod(X, model_class, contamination=0.1):
    model = model_class(contamination=contamination)
    model.fit(X)
    labels = model.labels_        # 0 = normal, 1 = outlier
    scores = model.decision_scores_  # skor anomali
    return labels, scores

- ABOD 
abod_labels, abod_scores = detect_outlier_pyod(X, ABOD)
df['abod_label'] = abod_labels
df['abod_score'] = abod_scores
abod_normal = df[df['abod_label'] == 0]
abod_outlier = df[df['abod_label'] == 1]
print("\nABOD Outlier Count:", len(abod_outlier))

- KNN 
knn_labels, knn_scores = detect_outlier_pyod(X, KNN)
df['knn_label'] = knn_labels
df['knn_score'] = knn_scores
knn_normal = df[df['knn_label'] == 0]
knn_outlier = df[df['knn_label'] == 1]
print("KNN Outlier Count:", len(knn_outlier))

- LOF 
lof_labels, lof_scores = detect_outlier_pyod(X, LOF)
df['lof_label'] = lof_labels
df['lof_score'] = lof_scores
lof_normal = df[df['lof_label'] == 0]
lof_outlier = df[df['lof_label'] == 1]
print("LOF Outlier Count:", len(lof_outlier))

- Tampilkan hasil
display(df.head())

Penjelasan:
Pada tahap ini dilakukan proses deteksi outlier menggunakan tiga metode berbeda dari library PyOD, yaitu ABOD (Angle-Based Outlier Detection), KNN (k-Nearest Neighbors), dan LOF (Local Outlier Factor). Pertama, dataset data_iris.xlsx dibaca dengan pandas, lalu apabila terdapat kolom kategori bernama Class, kolom tersebut diubah menjadi numerik menggunakan LabelEncoder agar dapat digunakan bersama fitur numerik lainnya. Selanjutnya hanya kolom-kolom numerik yang dipilih, karena algoritma deteksi outlier tidak dapat bekerja pada data kategorikal. Data numerik ini kemudian dijadikan input ke fungsi detect_outlier_pyod() yang dibuat khusus untuk menjalankan model PyOD. Fungsi ini bekerja dengan cara memanggil model yang ditentukan (ABOD, KNN, atau LOF), melatihnya pada data numerik, lalu menghasilkan dua keluaran, yaitu label yang menandakan apakah sebuah data dianggap normal (0) atau outlier (1), serta score yang menunjukkan tingkat keanehan data.

import matplotlib.pyplot as plt
import seaborn as sns

- Visualisasi Outlier dengan Matplotlib
- Use the original numerical features list for visualization
numerical_features_for_viz = df.select_dtypes(include=['float64', 'int64', 'object']).columns.tolist()
- Remove 'id' and outlier columns from numerical features for visualization
numerical_features_for_viz = [col for col in numerical_features_for_viz if '_outlier' not in col and col != 'id' and col != 'Class']


for feature in numerical_features_for_viz:
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=df, x='Class', y=feature, hue='Class', palette='viridis', s=50)

    - Highlight outliers based on IQR detection
    - Ensure the outlier column exists before trying to plot
    if feature + '_outlier' in df.columns:
        outliers_iqr = df[df[feature + '_outlier'] == 1]
        plt.scatter(outliers_iqr['Class'], outliers_iqr[feature], color='red', label='IQR Outlier', s=100, facecolors='none', edgecolors='red')

    - You can add other outlier detection results here if needed, e.g., for ABOD, KNN, LOF
    - For simplicity, let's add ABOD outliers as an example
    - outliers_abod = df[df['abod_label'] == 1]
    - plt.scatter(outliers_abod['Class'], outliers_abod[feature], color='blue', label='ABOD Outlier', s=100, facecolors='none', edgecolors='blue', marker='X')


    plt.title(f'Distribution of {feature} by Class with Outliers Highlighted')
    plt.xlabel('Class')
    plt.ylabel(feature)
    plt.legend()
    plt.grid(True)
    plt.show()